In [1]:
# default_exp scraper

# Scraper

> This module is used to scrape https://www.point2homes.com for all land available on Vancouver Island.

In [50]:
#hide
from nbdev.showdoc import *
from lxml.etree import HTML
from selenium import webdriver
from selenium.webdriver.common.by import By
import xmltodict as xd
from time import sleep
from lxml import etree
import pandas as pd

In [51]:
def read_land_column(lands):
    land_list = []
    for land in lands:
        data = xd.parse(etree.tostring(land))['div']['div']
        address = data[0]['@data-address']
        try:
            price = data[1]['@data-price']
            try:
                lot_size = data[2]['ul']['li'][0]['strong']
            except:
                print("Missing lot_size")
                lot_size = ''
            try:
                property_type = data[2]['ul']['li'][1]['#text']
            except:
                print("Missing property_type")
                property_type = ''
            link = data[4]['a']['@href']
            land_list.append({
                'address': address,
                'price':price,
                'lot_size':lot_size,
                'property_type':property_type,
                'link':link
            })
        except Exception as e:
            raise e
            print(f'Failed on {address}')
    return land_list

In [52]:
page_source = HTML(browser.page_source)

lands = page_source.xpath("//div[@class='item_information']")

lands = read_land_column(lands)

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=58533): Max retries exceeded with url: /session/dad648be-f16d-43d0-bbdd-017f546e28e1/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f4bf081e128>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [53]:
url = "https://www.point2homes.com/MX/Land-For-Sale-Maps.html?location=Mexico&search_mode=location&SelectedView=map_view&LocationGeoAreaId=33&ajax=1"
browser = webdriver.Firefox()
browser.get(url)

In [55]:
land_data = []

while(1):
    # Read Page
    page_source = HTML(browser.page_source)

    # Extract Land Column
    lands = page_source.xpath("//div[@class='item_information']")

    # Append data
    land_data = land_data + read_land_column(lands)
    
    # Report in.
    print(f"Successfully Scraped: {len(lands)}")

    # Go to next page
    next_button = browser.find_element(By.XPATH, "//a[starts-with(@class,'pager-next')]")
    if 'disabled' in next_button.get_attribute('class'):
        break
    else:
        print('Next Page')
        next_button.click()
        sleep(8)

Missing lot_size
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Successfully Scraped: 100
Next Page
Missing property_type
Missing property_type
Missing lot_size
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing lot_size
Missing property_type
Missing lot_size
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing lot_size
Missing property_type
Missing lot_size
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing property_type
Missing lot_size
Missing property_type
Missing property_type
Successfully Scraped: 100
Next Page
Missing lot_size


In [48]:
land_data

[[{'address': 'Lot 2 Las Colinas ',
   'price': '$675,000 USD',
   'lot_size': '0.354 ac',
   'property_type': 'Land',
   'link': '/MX/Vacant-Land-For-Sale/Baja-California-Sur/Los-Cabos/Tourist-Corridor/Cabo-del-Sol/Lot-2-Las-Colinas/85136368.html'},
  {'address': 'No address available ',
   'price': '$26,373,222 USD',
   'lot_size': '530.71 ac',
   'property_type': 'Land',
   'link': '/MX/Vacant-Land-For-Sale/Quintana-Roo/Punta-Maroma/No-address-available/82762008.html'},
  {'address': 'Boca del Alamo - Lote 6 ',
   'price': '$949,000 USD',
   'lot_size': '17 ac',
   'property_type': 'Land',
   'link': '/MX/Vacant-Land-For-Sale/Baja-California-Sur/Los-Cabos/Boca-del-Alamo-Lote-6/87109382.html'},
  {'address': 'Lote 5 de Diciembre ',
   'price': '$569,000 USD',
   'lot_size': '22,562.02',
   'property_type': 'Land',
   'link': '/MX/Vacant-Land-For-Sale/Jalisco/Puerto-Vallarta/5-de-Diciembre/Lote-5-de-Diciembre/89466621.html'},
  {'address': 'Jade Beach, Akumal, Riviera Maya ',
   'pric

In [136]:
df = pd.DataFrame(land_data)

In [137]:
import re

In [138]:
def extract_numbers(s):
    return float(''.join(re.findall("[-+]?\d*\.\d+|\d+", s)) or 0)

In [139]:
df['price_float'] = df['price'].apply(extract_numbers)

In [140]:
df['lot_size_float'] = df['lot_size'].apply(extract_numbers)

In [141]:
df['Dollar per Acre'] = df['price_float'] / df['lot_size_float']

In [142]:
df.head()

,address,price,lot_size,property_type,link,price_float,lot_size_float,Dollar per Acre
0,Harvey's Lot,"$360,000 USD",0.148 ac,Land,/MX/Vacant-Land-For-Sale/Quintana-Roo/Isla-Muj...,360000.0,0.148,2.432432e+06
1,Four Seasons Resort Condominiums,"$2,500,000 USD",,,/MX/Vacant-Land-For-Sale/Baja-California-Sur/L...,2500000.0,0.000,inf
2,"Ocean View Lot D-26, East Cape","$45,000 USD","19,977",Land,/MX/Vacant-Land-For-Sale/Baja-California-Sur/L...,45000.0,19977.000,2.252590e+00
3,Lote San Luis - Carr. Transpeninsular,"$6,923,279 USD",8 ac,Land,/MX/Vacant-Land-For-Sale/Baja-California-Sur/L...,6923279.0,8.000,8.654099e+05
4,KUYABEH,"$96,000 USD",2.471 ac,Land,/MX/Vacant-Land-For-Sale/Quintana-Roo/Tulum/KU...,96000.0,2.471,3.885067e+04


In [143]:
df = df.sort_values('Dollar per Acre')

In [144]:
df['link'] = 'https://www.point2homes.com' + df['link']

In [145]:
df = df[df['lot_size_float'] > 0]

In [146]:
df['Dollar per Acre'] = df['Dollar per Acre'].apply(lambda x: str(round(x, 4)))

In [148]:
df.head(100).to_csv('MexicoTop100.csv', index=False)

In [81]:
df.to_csv('VancouverIsland.csv', index=False)

In [ ]:
len(pages)extract_numbers

121

In [ ]:
df = pd.DataFrame(land_list)

In [ ]:
df.sort_values('price_float').to_csv('VancouverIsland-Cheapest.csv',index=False)

In [ ]:
df.iloc[92].values

array(['LT 38 RUXTON ISLAND ', '$75,000 CAD', '2.15 ac', 'Land',
       '/CA/Vacant-Land-For-Sale/BC/Vancouver-Island/Cowichan-Valley/LT-38-RUXTON-ISLAND/88932209.html'],
      dtype=object)

In [ ]:
next_page = browser.find_element(By.XPATH, "//input[starts-with(@name,'username')]")
password = browser.find_element(By.XPATH, "//input[starts-with(@name,'password')]")

username.send_keys("Carole@impactisi.com")
password.send_keys("password123")
browser.find_element(By.XPATH, "//input[starts-with(@name,'Login')]").click()

In [ ]:
pager-next

In [ ]:
# price, ac, price/ac, datetime, id, area

In [ ]:
1038 / 100

10.38

In [ ]:
item = {
    'address':
    'price':
    'characteristics':
    'labels':
    'detail_link'
}

SyntaxError: invalid syntax (<ipython-input-111-5589d66c2c9c>, line 3)

In [ ]:
import pandas as pd

In [ ]:
from lxml import etree

In [ ]:
etree.tostring(tree)

b'<div class="item_information">\n        <div class="item_address" data-url="/CA/Vacant-Land-For-Sale/BC/Nanoose-Bay/LOT-1-Oak-Leaf-Drive/82258309.html" data-tracking="" data-address="LOT 1 Oak Leaf Drive ">\n            <div class="address-container">\n                <span>LOT 1 Oak Leaf Drive </span>\n            </div>\n\n        </div>\n<div class="price " data-price="$799,000 CAD">\n    <span class="green">\n            <span> $799,000 CAD</span>\n    </span>\n\n</div>\n        <div class="characteristics-cnt">\n\n\n<ul>\n                    <li data-label="Lot Size" class="ic-lotsize">\n            <strong>0.842 ac</strong> <span class="gray">Lot Size</span>\n        </li>\n            <li class="property-type ic-proptype">\n            Land\n        </li>\n</ul>\n\n\n        </div>\n        <div class="labels">\n            <div class="featured-icon" title="Featured"/>\n\n        </div>\n        <div class="lnk_details">\n            <a rel="" href="/CA/Vacant-Land-For-Sale/BC

In [ ]:
etree.tostringlist(tree)

[b'<div class="item_information">\n        <div class="item_address" data-url="/CA/Vacant-Land-For-Sale/BC/Nanoose-Bay/LOT-1-Oak-Leaf-Drive/82258309.html" data-tracking="" data-address="LOT 1 Oak Leaf Drive ">\n            <div class="address-container">\n                <span>LOT 1 Oak Leaf Drive </span>\n            </div>\n\n        </div>\n<div class="price " data-price="$799,000 CAD">\n    <span class="green">\n            <span> $799,000 CAD</span>\n    </span>\n\n</div>\n        <div class="characteristics-cnt">\n\n\n<ul>\n                    <li data-label="Lot Size" class="ic-lotsize">\n            <strong>0.842 ac</strong> <span class="gray">Lot Size</span>\n        </li>\n            <li class="property-type ic-proptype">\n            Land\n        </li>\n</ul>\n\n\n        </div>\n        <div class="labels">\n            <div class="featured-icon" title="Featured"/>\n\n        </div>\n        <div class="lnk_details">\n            <a rel="" href="/CA/Vacant-Land-For-Sale/B

In [ ]:
pd.read_html(tree)

/usr/lib/python3.6/urllib/parse.py:107: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  return tuple(x.decode(encoding, errors) if x else '' for x in args)


TypeError: Cannot read object of type '_Element'

In [ ]:
tree = lands[0]

In [ ]:
price = tree.xpath("//div[@class='price']")


In [ ]:
price

[<Element div at 0x7fb8ac045a88>]

In [ ]:
e = price[0]

In [ ]:
e.text

'$47,900 CAD'

In [ ]:
address = tree.xpath("//div[@class='item_address']")


In [ ]:
address[0].attrib['data-address']

'LOT 1 Oak Leaf Drive '

In [ ]:
data-address

In [ ]:
descriptions = tree.xpath("//div[@class='info member-info']")
links = tree.xpath("//a[@class='learn-more']")

In [ ]:
lands

[<Element div at 0x7fb8ac0d0408>,
 <Element div at 0x7fb8ac0d0d08>,
 <Element div at 0x7fb8ac0f57c8>,
 <Element div at 0x7fb8ac045448>,
 <Element div at 0x7fb8ac045488>,
 <Element div at 0x7fb8ac045508>,
 <Element div at 0x7fb896df03c8>,
 <Element div at 0x7fb896df0408>,
 <Element div at 0x7fb896df0448>,
 <Element div at 0x7fb8ac0454c8>,
 <Element div at 0x7fb896df0488>,
 <Element div at 0x7fb896df04c8>,
 <Element div at 0x7fb896df0508>,
 <Element div at 0x7fb896df0548>,
 <Element div at 0x7fb896df0588>,
 <Element div at 0x7fb896df05c8>,
 <Element div at 0x7fb896df0608>,
 <Element div at 0x7fb896df0648>,
 <Element div at 0x7fb896df0688>,
 <Element div at 0x7fb896df06c8>,
 <Element div at 0x7fb896df0708>,
 <Element div at 0x7fb896df0748>,
 <Element div at 0x7fb896df0788>,
 <Element div at 0x7fb896df07c8>,
 <Element div at 0x7fb896df0808>,
 <Element div at 0x7fb896df0848>,
 <Element div at 0x7fb896df0888>,
 <Element div at 0x7fb896df08c8>,
 <Element div at 0x7fb896df0908>,
 <Element div 